In [28]:
import numpy as np
import cv2
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt


from os import listdir

In [29]:
import tkinter as tk
from tkinter import Message ,Text
import cv2,os
import shutil
import csv
import numpy as np
import PIL
from PIL import Image, ImageTk
import pandas as pd
import datetime
import time
import tkinter.ttk as ttk
import tkinter.font as font
import re
import smtplib

#subject import module
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import email, smtplib, ssl, csv

from email import encoders
from email.mime.base import MIMEBase

In [30]:
color = (67,67,67)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [31]:
from openpyxl import Workbook
import datetime

In [32]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [33]:

def loadVggFaceModel():
	model = Sequential()
	model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
	model.add(Convolution2D(64, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(64, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(128, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(128, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(256, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(ZeroPadding2D((1,1)))
	model.add(Convolution2D(512, (3, 3), activation='relu'))
	model.add(MaxPooling2D((2,2), strides=(2,2)))

	model.add(Convolution2D(4096, (7, 7), activation='relu'))
	model.add(Dropout(0.5))
	model.add(Convolution2D(4096, (1, 1), activation='relu'))
	model.add(Dropout(0.5))
	model.add(Convolution2D(2622, (1, 1)))
	model.add(Flatten())
	model.add(Activation('softmax'))
	from keras.models import model_from_json
	model.load_weights('vgg_face_weights.h5')
	
	vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
	
	return vgg_face_descriptor

In [34]:
model = loadVggFaceModel()

employee_pictures = "deepika/"

employees = dict()

for file in listdir(employee_pictures):
	employee, extension = file.split(".")
	employees[employee] = model.predict(preprocess_image('deepika/%s.jpg' % (employee)))[0,:]
	

def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
cap = cv2.VideoCapture(0) 
c = 0

In [35]:
book=Workbook()
sheet=book.active
now= datetime.datetime.now()
today=now.day
month=now.month

In [37]:
print("employee representations retrieved successfully")
#if(is_number(Id)employees.isalpha()):
while(True):
    ret, img = cap.read()
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    eyes = eye_cascade.detectMultiScale(img, 1.3, 5)
    for (x,y,w,h) in faces:
        k = 1
        if w > 130: 
            cv2.rectangle(img, (x,y), (x+w,y+h), (255, 0, 0) , 3)
            detected_face = img[int(y):int(y+h), int(x):int(x+w)] 
            detected_face = cv2.resize(detected_face, (224, 224)) 
            img_pixels = image.img_to_array(detected_face)
            img_pixels = np.expand_dims(img_pixels, axis = 0)
            img_pixels /= 127.5
            img_pixels -= 1
            for (ex,ey,ew,eh) in eyes:
                k=0
                cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(0,0,255),2)

            captured_representation = model.predict(img_pixels)[0,:]

            found = 0
            for i in employees:
                employee_name = i
                representation = employees[i]

                similarity = findCosineSimilarity(representation, captured_representation)
                if(similarity < 0.30):
                    cv2.putText(img, employee_name, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2)
                    found= 5
                    #if(k <= 0):
                        #c=c+1
                        #cv2.putText(img, employee_name+"+ Present",(int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2)
                    #else:
                     #   cv2.putText(img, employee_name+"+ fake",(int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0) , 2)
                    #found = 5
                    break

            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,2)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,2)
            if(found == 0): 
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            
    cv2.imshow('img',img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

row = [employee_name , "present", now.month, datetime.datetime.now()]#row format 
with open('StudentDetails\StudentDetails.csv','a+') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(row)
    csvFile.close()
        
print("email sent\n")

employee representations retrieved successfully
email sent

